In [2]:
import pandas as pd
import pyodbc
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [3]:
s = "DSN=PRDDW; UID={}; PWD={}".format('vigrose', 'slytherin10946')
AMAEDW = pyodbc.connect(s)

In [4]:
sql_query_1 = \
    """ 
    SELECT DISTINCT
    P.PARTY_ID, 
    P.KEY_VAL AS ME
    FROM 
    AMAEDW.PARTY_KEY P
    WHERE  
    P.KEY_TYPE_ID = 18 
    AND 
    P.ACTIVE_IND = 'Y'
    """
ME = pd.read_sql(con=AMAEDW, sql=sql_query_1)

In [5]:
sql_query_2 = \
    """ 
    SELECT DISTINCT
    P.PARTY_ID, 
    P.KEY_VAL AS NPI
    FROM 
    AMAEDW.PARTY_KEY P
    WHERE  
    P.KEY_TYPE_ID = 38 
    AND 
    P.ACTIVE_IND = 'Y'
    """
NPI = pd.read_sql(con=AMAEDW, sql=sql_query_2)

In [6]:
npi_to_me = pd.merge(NPI, ME, on='PARTY_ID')

In [7]:
DF_1 = pd.read_csv('DHC_data/ContactInfo_AK.csv')
DF_2 = pd.read_csv('DHC_data/ContactInfo_AZALAR.csv')
DF_3 = pd.read_csv('DHC_data/ContactInfo_CACOCTDE.csv')
DF_4 = pd.read_csv('DHC_data/ContactInfo_FL-KY.csv')
DF_5 = pd.read_csv('DHC_data/ContactInfo_LA-NH.csv')
DF_6 = pd.read_csv('DHC_data/ContactInfo_NJ-OR.csv')
DF_7 = pd.read_csv('DHC_data/ContactInfo-PA-TX.csv')
DF_8 = pd.read_csv('DHC_data/ContactInfo_UT-WY.csv')

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
all_dhc = pd.concat([DF_1,DF_2,DF_3,DF_4,DF_5,DF_6,DF_7,DF_8])

In [11]:
npi_to_me = npi_to_me[['NPI','ME']]
dhc_ = pd.merge(npi_to_me, all_dhc, on='NPI', how = 'right')

In [10]:
all_dhc['NPI'] = all_dhc['NPI'].astype(str)

In [15]:
dhc_.to_csv('DHC_data/DHC_data.csv',index=False)

In [31]:
results = pd.read_csv('Add_Google_Results.csv')

In [45]:
results.PHYSICIAN_ME_NUMBER = results.PHYSICIAN_ME_NUMBER.astype('int64').astype(str)

In [64]:
all_data = pd.merge(results, dhc_, on='ME', suffixes =['_MF','_DHC'])

In [57]:
nums =[]
for num in list(results['PHYSICIAN_ME_NUMBER']):
    if len(num)==10:
        num = '0'+num
    elif len(num)==9:
        num = '00'+num
    elif len(num)==8:
        num = '000'+num
    nums.append(num)

In [58]:
results['ME']=nums

In [97]:
all_data = all_data.rename(columns ={'Additional Office Locations':'Add_Offices', 'Phone Number':'Phone_Number', 'Zip Code':'Zipcode_DHC', 'First Name':'First', 'Physician Name':'Name_DHC',
       'Middle Name':'Middle', 'Primary Specialty':'Spec', 'Primary Hospital Affiliation':'Affiliation',
       'Last Name':'Last'})

In [73]:
all_data = all_data.fillna('None')

In [96]:
match_count = 0
add_match_count = 0 
city_count = 0
for row in all_data.itertuples():
    add_match =False
    if str(row.Phone_Number.replace('.','')) == str(row.OFFICE_TELEPHONE):
        match_count += 1
    add_dhc = row.Address_DHC.upper()
    add_mf = row.OFFICE_ADDRESS_LINE_2
    if add_dhc in add_mf or add_mf in add_dhc:
        add_match = True
    elif fuzz.ratio(add_dhc,add_mf)>70:
        add_match =True
    if add_match == True:
        add_match_count += 1
    if row.OFFICE_ADDRESS_CITY == row.City_DHC.upper():
        city_count += 1
#     else:
#         print(f'{add_dhc} does not match to {add_mf}')
print(match_count)
print(add_match_count)
print(city_count)

172
244
297


In [77]:
all_data[['OFFICE_ADDRESS_LINE_2','Address_DHC']]

,OFFICE_ADDRESS_LINE_2,Address_DHC
0,1215 21ST AVE S,1310 24th Ave N
1,1807 W SLAUGHTER LN STE 490,1807 W Slaughter Ln
2,15905 UNION TPKE,2639 Atlantic Ave
3,3734 7TH ST STE 22,600 W Lake Cook Rd
4,201 MONROE ST STE 1382,201 Monroe St
...,...,...
452,6511 SPRING BROOK AVE,6511 Spring Brook Ave
453,3615 19TH ST,3615 19th St
454,313 CALLE MANUEL DOMENECH STE 101,313 Avenue Domenech
455,912 S WOOD ST,525 E Congress Pkwy


In [92]:
fuzz.ratio('39TH AND CHESTNUT ST','3819 CHESTNUT ST')

78

In [82]:
fuzz.ratio('315 W 49TH ST','315A W 49TH ST')

96

In [106]:
all_data.columns

Index(['Unnamed: 0', 'PHYSICIAN_ME_NUMBER', 'PHYSICIAN_FIRST_NAME',
       'PHYSICIAN_MIDDLE_NAME', 'PHYSICIAN_LAST_NAME', 'SUFFIX', 'DEGREE',
       'OFFICE_ADDRESS_LINE_1', 'OFFICE_ADDRESS_LINE_2', 'OFFICE_ADDRESS_CITY',
       'OFFICE_ADDRESS_STATE', 'OFFICE_ADDRESS_ZIP', 'OFFICE_TELEPHONE',
       'Number', 'Name', 'Address_MF', 'Zipcode', 'City_MF', 'State_MF',
       'PhoneType', 'QualityScore', 'Date', 'Notes', 'Provider', 'SICCode',
       'SICDesc', 'Label', 'Google_Name', 'Google_Address', 'Google_Types',
       'Google_Names', 'Google_Addresses', 'ME', 'NPI', 'Add_Offices',
       'Phone_Number', 'Zipcode_DHC', 'State_DHC', 'City_DHC', 'Address_DHC',
       'First', 'Name_DHC', 'Middle', 'Spec', 'Affiliation', 'Last',
       'Address1'],
      dtype='object')

In [126]:
match_count = 0
add_match_count = 0 
city_count = 0
for row in all_data.itertuples():
    add_match =False
    if str(row.Phone_Number.replace('.','')) != str(row.OFFICE_TELEPHONE):
        add_dhc = row.Address_DHC.upper()
        add_gfi = row.Google_Address.split(', ')[0].upper()
        if add_gfi != 'NONE':
            match_count += 1
            adds = row.Google_Addresses.replace('"',"'").replace("['","").replace("']","")
            all_adds = adds.split("', '")
            for add in all_adds:
                add_gfi = add.split(', ')[0].upper()
                if add_dhc in add_gfi or add_gfi in add_dhc:
                    add_match = True
                    print(f'{add_dhc} matches to {add_gfi}')
                    print('----')
                    break
                elif fuzz.ratio(add_dhc,add_gfi)>87:
                    add_match =True
                    print(f'{add_dhc} matches to {add_gfi}')
                    print('----')
                    break
            if add_match == True:
                
                add_match_count += 1
#             if add_match == False:
#                 print(f'{add_dhc} does not match to {add_gfi}')
#                 print('----')
#     if row.OFFICE_ADDRESS_CITY == row.City_DHC.upper():
#         city_count += 1
#     else:
#         print(f'{add_dhc} does not match to {add_mf}')
print(match_count)
print(add_match_count)
print(add_match_count/match_count)

2900 12TH AVE N matches to 2900 12TH AVE N # 205W
----
915 OLD FERN HILL RD matches to 915 OLD FERN HILL RD
----
25 HACKETT BLVD matches to 62 HACKETT BLVD
----
795 EL CAMINO REAL matches to 795 EL CAMINO REAL
----
2630 HOLME AVE matches to 2630 HOLME AVE SUITE 200
----
1519 E 6TH ST matches to 1519 E 6TH ST
----
4315 DIPLOMACY DR matches to 4320 DIPLOMACY DR
----
282 THE GRN matches to 282 THE GREEN
----
80 SHUNPIKE RD matches to 80 SHUNPIKE RD #301
----
72 5TH ST SE matches to 72 5TH ST SE SUITE A
----
1250 E MARSHALL ST matches to 1250 E MARSHALL ST
----
4940 EASTERN AVE matches to 4940 EASTERN AVE
----
1600 ROCKLAND RD matches to 1600 ROCKLAND RD
----
428 BILTMORE AVE matches to 428 BILTMORE AVE
----
735 HASKINS RD matches to 735 HASKINS RD # G
----
11123 PARKVIEW PLAZA DR matches to 11123 PARKVIEW PLAZA DR #101
----
1000 N OAK AVE matches to 1000 N OAK AVE
----
5841 S MARYLAND AVE matches to 5841 S MARYLAND AVE
----
1 CHILDREN'S WAY LITTLE matches to 1 CHILDREN'S WAY
----
990 SOUT

In [112]:
all_data['Google_Addresses']

0                                                     []
1                                                     []
2                                                     []
3      ['600 W Lake Cook Rd, Buffalo Grove, IL 60089,...
4                                                     []
                             ...                        
452    ['6511 Spring Brook Ave, Rhinebeck, NY 12572, ...
453             ['3615 19th St, Lubbock, TX 79410, USA']
454                                                   []
455    ['22285 N Pepper Rd #401, Barrington, IL 60010...
456    ['315 W 49th St A, Hialeah, FL 33012, USA', '3...
Name: Google_Addresses, Length: 457, dtype: object

In [125]:
fuzz.ratio('2900 12TH AVE N','2800 10TH AVE N')

87

In [127]:
all_data.to_csv('add_dhc.csv',index=False)

In [133]:
false_neg =0
false_pos =0
true_pos =0
true_neg = 0

for row in all_data.itertuples():
    add_match =False
    phone_match = False
    if str(row.Phone_Number.replace('.','')) == str(row.OFFICE_TELEPHONE):
        phone_match = True
    add_dhc = row.Address_DHC.upper()
    add_mf = row.OFFICE_ADDRESS_LINE_2
    if add_dhc in add_mf or add_mf in add_dhc:
        add_match = True
    elif fuzz.ratio(add_dhc,add_mf)>70:
        add_match =True
    if row.Label == 1:
        if add_match == True:
            false_neg += 1
        else:
            true_neg += 1       
    if row.Label == 0:
        if add_match == True:
            true_pos += 1
        else:
            false_pos += 1  

In [134]:
true_pos

132

In [135]:
true_neg

124

In [136]:
false_pos

89

In [138]:
all_data.columns

Index(['Unnamed: 0', 'PHYSICIAN_ME_NUMBER', 'PHYSICIAN_FIRST_NAME',
       'PHYSICIAN_MIDDLE_NAME', 'PHYSICIAN_LAST_NAME', 'SUFFIX', 'DEGREE',
       'OFFICE_ADDRESS_LINE_1', 'OFFICE_ADDRESS_LINE_2', 'OFFICE_ADDRESS_CITY',
       'OFFICE_ADDRESS_STATE', 'OFFICE_ADDRESS_ZIP', 'OFFICE_TELEPHONE',
       'Number', 'Name', 'Address_MF', 'Zipcode', 'City_MF', 'State_MF',
       'PhoneType', 'QualityScore', 'Date', 'Notes', 'Provider', 'SICCode',
       'SICDesc', 'Label', 'Google_Name', 'Google_Address', 'Google_Types',
       'Google_Names', 'Google_Addresses', 'ME', 'NPI', 'Add_Offices',
       'Phone_Number', 'Zipcode_DHC', 'State_DHC', 'City_DHC', 'Address_DHC',
       'First', 'Name_DHC', 'Middle', 'Spec', 'Affiliation', 'Last',
       'Address1'],
      dtype='object')

In [ ]:
#first test GetPhoneInfo

for row in all_data.itertuples():
    if row.Address_MF.upper()

In [143]:
dhc_[dhc_.ME.isna()]

,NPI,ME,Additional Office Locations,Phone Number,Zip Code,State,City,Address,First Name,Physician Name,Middle Name,Primary Specialty,Primary Hospital Affiliation,Last Name,Address1
934625,1780063727,NaN,NaN,907.580.3205,99506,AK,Jber,5955 Zeamer Ave,Caleb,Caleb Baxter,Spencer,Anesthesiology - Anesthesiologist,NaN,Baxter,NaN
934626,1467611699,NaN,NaN,907.486.9870,99615,AK,Kodiak,3449 E Rezanof Dr,Georgia,Georgia Blain,J,General Practice,NaN,Blain,NaN
934627,1811359896,NaN,NaN,907.729.5011,99654,AK,Wasilla,1001 S Knik Goose Bay Rd,Alexandria,Alexandria Boley,Paige,Pediatric Medicine,NaN,Boley,NaN
934628,1669660056,NaN,NaN,907.714.4444,99669,AK,Soldotna,250 Hospital Pl,Kyland,Kyland Burden,I,Emergency Medicine,"Coon Memorial Hospital (Dalhart, TX)",Burden,NaN
934629,1639306947,NaN,NaN,907.580.3205,99506,AK,Jber,5955 Zeamer Ave,Rebecca,Rebecca Burns,K,Obstetrics/Gynecology,NaN,Burns,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
969967,1407142615,NaN,NaN,206.288.6956,98109,WA,Seattle,825 Eastlake Ave E,Maria,Maria Zudaire,Teresa,Oncology - Hematology/Oncology,NaN,Zudaire,NaN
969968,1912464595,NaN,NaN,703.766.2220,20190,VA,Reston,1830 Town Center Dr Ste 410,Arnela,Arnela Zukanovic,NaN,Dermatology,NaN,Zukanovic,NaN
969969,1356737209,NaN,NaN,920.969.7900,54956,WI,Neenah,130 2nd St,Casandra,Casandra Zummallen,Lynn,Pediatric Medicine,ThedaCare Regional Medical Center - Appleton (...,Zummallen,NaN
969970,1699161901,NaN,NaN,608.263.5660,53792,WI,Madison,Uw Hospital And Clinics,Luke,Luke Zurbriggen,NaN,Internal Medicine,"University Hospital (Madison, WI)",Zurbriggen,NaN


In [146]:
dhc_ = dhc_.rename(columns ={'Additional Office Locations':'Additional_Offices', 'Phone Number':'Phone_Number', 'Zip Code':'Zipcode', 'First Name':'First', 'Physician Name':'Name',
       'Middle Name':'Middle', 'Primary Specialty':'Specialty', 'Primary Hospital Affiliation':'Affiliation',
       'Last Name':'Last'})

In [ ]:
dhc_=dhc_.fillna('None')

In [160]:
phones =[]
zipcodes =[]
middles =[]
firsts=[]
lasts=[]
for row in dhc_.itertuples():
    phones.append(row.Phone_Number.replace('.',''))
    zipcodes.append(str(row.Zipcode).replace('=','').replace('"',''))
    if row.Middle == 'NONE':
        middles.append('None')
    else:    
        middles.append(row.Middle.upper())
    firsts.append(row.First.upper())
    lasts.append(row.Last.upper())

In [161]:
dhc_['Zipcode']=zipcodes
dhc_['Middle']=middles
dhc_['Phone_Number']=phones
dhc_['First']=firsts
dhc_['Last']=lasts

In [168]:
all_data.columns

Index(['Unnamed: 0', 'PHYSICIAN_ME_NUMBER', 'PHYSICIAN_FIRST_NAME',
       'PHYSICIAN_MIDDLE_NAME', 'PHYSICIAN_LAST_NAME', 'SUFFIX', 'DEGREE',
       'OFFICE_ADDRESS_LINE_1', 'OFFICE_ADDRESS_LINE_2', 'OFFICE_ADDRESS_CITY',
       'OFFICE_ADDRESS_STATE', 'OFFICE_ADDRESS_ZIP', 'OFFICE_TELEPHONE',
       'Number', 'Name', 'Address_MF', 'Zipcode', 'City_MF', 'State_MF',
       'PhoneType', 'QualityScore', 'Date', 'Notes', 'Provider', 'SICCode',
       'SICDesc', 'Label', 'Google_Name', 'Google_Address', 'Google_Types',
       'Google_Names', 'Google_Addresses', 'ME', 'NPI', 'Add_Offices',
       'Phone_Number', 'Zipcode_DHC', 'State_DHC', 'City_DHC', 'Address_DHC',
       'First', 'Name_DHC', 'Middle', 'Spec', 'Affiliation', 'Last',
       'Address1'],
      dtype='object')

In [182]:
good_phones = 0
good_adds = 0
bad_phones = 0
bad_adds = 0
false_pos =0
false_neg =0

for row in all_data.itertuples():
    dhc_add_match =False
    phone_match = False
    gfi_add_match = False
    if str(row.Phone_Number.replace('.','')) == str(row.OFFICE_TELEPHONE):
        phone_match = True
    add_dhc = row.Address_DHC.upper()
    add_mf = row.OFFICE_ADDRESS_LINE_2
    add_gfi = row.Address_MF
    if add_dhc in add_mf or add_mf in add_dhc:
        dhc_add_match = True
    elif fuzz.ratio(add_dhc,add_mf)>70:
        dhc_add_match =True
    if add_gfi in add_mf or add_mf in add_gfi:
        gfi_add_match = True
    elif fuzz.ratio(add_gfi,add_mf)>70:
        gfi_add_match =True
        
#     if gfi_add_match == True & phone_match ==True:
#         good_phones += 1
#     if gfi_add_match == False & phone_match ==False:
#         bad_phones += 1
        
    if gfi_add_match == True & dhc_add_match ==False:
        false_pos += 1
    if gfi_add_match == False & dhc_add_match ==True:
        false_neg += 1
        
    if gfi_add_match == False & dhc_add_match ==False:
        bad_adds += 1
    if gfi_add_match == True & dhc_add_match ==True:
        good_adds += 1

In [186]:
false_neg

0

In [181]:
false_pos

210

In [198]:
good_phones = 0
good_adds = 0
bad_phones = 0
bad_adds = 0
false_pos =0
false_neg =0

for row in g_data.itertuples():
    dhc_add_match =False
    phone_match = False
    gfi_add_match = False
    if str(row.Phone_Number.replace('.','')) == str(row.OFFICE_TELEPHONE):
        phone_match = True
    add_dhc = row.Address_DHC.upper()
    add_mf = row.OFFICE_ADDRESS_LINE_2
    
    if phone_match == True:
    if add_dhc in add_mf or add_mf in add_dhc:
        dhc_add_match = True
    elif fuzz.ratio(add_dhc,add_mf)>70:
        dhc_add_match =True

    adds = row.Google_Addresses.replace('"',"'").replace("['","").replace("']","")
    all_adds = adds.split("', '")
    for add in all_adds:
        add_gfi = add.split(', ')[0].upper()
        if add_mf in add_gfi or add_gfi in add_mf:
            gfi_add_match = True
            break
        elif fuzz.ratio(add_mf,add_gfi)>87:
            gfi_add_match =True
            break
        
    if gfi_add_match == True & phone_match ==True:
        good_phones += 1
    if gfi_add_match == False & phone_match ==False:
        bad_phones += 1
        
    if gfi_add_match == True & phone_match ==False:
        false_pos += 1
    if gfi_add_match == False & phone_match ==True:
        false_neg += 1
        
#     if gfi_add_match == False & dhc_add_match ==False:
#         bad_adds += 1
#     if gfi_add_match == True & dhc_add_match ==True:
#         good_adds += 1

In [190]:
g_data = all_data[all_data.Google_Address!='None']

In [202]:
false_neg

0

In [205]:
phone_labels = []
address_labels = []
phone_true = []
add_true = []

for row in all_data.itertuples():
    dhc_add_match =False
    phone_match = False
    gfi_add_match = False
    goog_add_match = 'inconclusive'
    if str(row.Phone_Number.replace('.','')) == str(row.OFFICE_TELEPHONE):
        phone_match = True
    add_dhc = row.Address_DHC.upper()
    add_mf = row.OFFICE_ADDRESS_LINE_2
    add_gfi = row.Address_MF
    
    if add_dhc in add_mf or add_mf in add_dhc:
        dhc_add_match = True
    elif fuzz.ratio(add_dhc,add_mf)>70:
        dhc_add_match =True
        
    if add_gfi in add_mf or add_mf in add_gfi:
        gfi_add_match = True
    elif fuzz.ratio(add_gfi,add_mf)>70:
        gfi_add_match =True
        
    if row.Google_Address != 'None':
        goog_add_match = False
        adds = row.Google_Addresses.replace('"',"'").replace("['","").replace("']","")
        all_adds = adds.split("', '")
        for add in all_adds:
            add_goog = add.split(', ')[0].upper()
            if add_mf in add_goog or add_goog in add_mf:
                goog_add_match = True
                break
            elif fuzz.ratio(add_mf,add_goog)>87:
                goog_add_match =True
                break
    if gfi_add_match == True:
        phone = True
        address = True
    elif goog_add_match == False:
        phone = False
        address = False
    elif goog_add_match == True:
        phone = True
        address = True
    elif gfi_add_match == False:
        phone = False
        address = False
    phone_labels.append(phone)
    address_labels.append(address)
    phone_true.append(phone_match)
    add_true.append(dhc_add_match)

In [211]:
preds = pd.DataFrame({'Phone_Label': phone_labels, 'Phone_Truth': phone_true, 'Address_Label': address_labels, 'Address_Truth':add_true})

In [213]:
count =0
for row in preds.itertuples():
    if row.Address_Label == row.Address_Truth:
        count += 1
print(count)

305
